In [1]:
import optuna
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Subset
from torchvision.datasets import MNIST
import random

In [2]:
# Definir el modelo de red neuronal
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleNN, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [3]:
# Load the MNIST dataset and define the DataLoader
def load_mnist():
    # Cargar el conjunto de datos MNIST y definir el DataLoader
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])
    full_train_dataset = MNIST(root='./data', train=True, download=True, transform=transform)
    subset_indices = random.sample(range(len(full_train_dataset)), k=1000)  # Use a subsample of 1000 samples
    train_dataset = Subset(full_train_dataset, subset_indices)
    return DataLoader(train_dataset, batch_size=64, shuffle=True)
    

In [4]:
# Definir la función de pérdida y el optimizador
def train(model, optimizer, criterion, train_loader, device):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs.view(-1, 28*28))
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item() * inputs.size(0)
    return running_loss / len(train_loader.dataset)

In [5]:
def objective(trial):
    # Definir los hiperparámetros a optimizar
    hidden_size = trial.suggest_int('hidden_size', 16, 128)
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-5, 1e-1)
    
    # Definir el dispositivo (CPU o GPU)
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(torch.cuda.is_available())
 
    # Crear una instancia del modelo
    model = SimpleNN(input_size=28*28, hidden_size=hidden_size, output_size=10).to(device)
    
    # Definir la función de pérdida y el optimizador
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    
    # Entrenar el modelo
    for epoch in range(10):
        train_loss = train(model, optimizer, criterion, train_loader, device)
    
    # Devolver la pérdida de validación para la evaluación de Optuna
    return train_loss


In [6]:
# Load the data once
train_loader = load_mnist()
# Ejecutar la optimización de hiperparámetros con Optuna


study = optuna.create_study(direction='minimize', storage="sqlite:///db.sqlite3v",study_name= "ejemplo") # Specify the storage URL here.
        
study.optimize(objective, n_trials=10)

DuplicatedStudyError: Another study with name 'ejemplo' already exists. Please specify a different name, or reuse the existing one by setting `load_if_exists` (for Python API) or `--skip-if-exists` flag (for CLI).

In [ ]:
# Mostrar los resultados de la optimización
print("Best trial:")
trial = study.best_trial
print("  Value: {}".format(trial.value))
print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

Best trial:
  Value: 0.19590255427360534
  Params: 
    hidden_size: 22
    learning_rate: 0.06819375415930178
